In [ ]:
# import necessary packages*+--
import os

import keras
import mne
import numpy as np
import tensorflow as tf
import tensorflow.keras
from keras.layers import (LSTM, Activation, BatchNormalization, Conv2D, Dense,
                          Dropout, Flatten, Reshape)
from keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import (LSTM, Activation, BatchNormalization,
                                     Conv2D, Dense, Dropout, Flatten, Reshape)
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

os.chdir('D:/EEG-ESRT/datasets/EEGMMIDB/')

SList = ['%03d' % i for i in range(1, 110)]
RList = ['%02d' % i for i in range(1, 15)]
Lists = []
for j in SList:
    name = 's' + str(j)
    name = np.zeros((64, 160))
    name = np.expand_dims(name, axis=0)
    for i in RList:
        file = os.getcwd() + '/S' + str(j) + '/S' + str(j) + 'R' + str(
            i) + '.edf'
        data = mne.io.read_raw_edf(file)
        raw_data = data.get_data()

        # 1 sec parts
        for k in range(int((np.size(raw_data, 1)) / 160)):
            name = np.concatenate(
                (name,
                 np.expand_dims(raw_data[:, (k) * 160:(k + 1) * 160], axis=0)),
                axis=0)

        # free up memory
        path = []
        cwd = os.getcwd()
        file = os.getcwd() + '/S' + str(j) + '/S' + str(j) + 'R' + str(
            i) + '.edf'
        path.append(os.path.join(cwd, file))
        os.remove(path[0])
    name = name[1:np.size(name, 0), :, :]
    subj = 'S' + str(j)
    np.save(subj, name)

# Selecting Channels
SList = ['%03d' % i for i in range(1, 110)]
RList = ['%02d' % i for i in range(1, 15)]

# For 32 Channels
channels32 = [
    0, 2, 3, 4, 6, 8, 10, 12, 14, 16, 17, 18, 20, 21, 22, 23, 26, 29, 31, 33,
    35, 37, 40, 41, 46, 48, 50, 52, 54, 57, 60, 62
]
for j in SList:
    name = 'S' + str(j) + '.npy'
    a = np.load(name)
    a = a[:, channels32, :]
    subject = 's' + str(j)
    np.save(subject, a)

# Assigning p% of data for training
SList = ['%03d' % i for i in range(1, 110)]
RList = ['%02d' % i for i in range(1, 15)]
p = 0.9
for j in SList:
    name = 's' + str(j) + '.npy'
    a = np.load(name)
    shuffle = list(range(np.size(a, 0)))
    np.random.shuffle(shuffle)
    a = a[shuffle, :, :]
    num_tr = int(p * np.size(a, 0))
    x_tr = a[0:num_tr, :, :]
    x_te = a[num_tr + 1:np.size(a, 0), :, :]
    subject_tr = 's' + str(j) + '_train'
    subject_te = 's' + str(j) + '_test'
    np.save(subject_tr, x_tr)
    np.save(subject_te, x_te)
    print(j)

import numpy as np
from tqdm import tqdm

SList = ['%03d' % i for i in range(1, 110)]
RList = ['%02d' % i for i in range(1, 15)]

# note that n_ch should be equal to what you selected in channel selection step!
# here we chose 4
n_ch = 32
y_train = np.zeros((1, 1))
x_train = np.zeros((1, n_ch, 160))
print('Train Dataset----------------------------------')
for j in tqdm(SList):
    name = 's' + str(j) + '_train.npy'
    a = np.load(name)
    #print(a.shape)
    x_train = np.concatenate((x_train, a), axis=0)
    y = (int(j) - 1) * np.ones((np.size(a, 0), 1))
    y_train = np.concatenate((y_train, y))
    # print(j)
    del a
    del y

print('Test Dataset----------------------------------')
# note that n_ch should be equal to what you selected in channel selection step!
# here we chose 4
y_test = np.zeros((1, 1))
x_test = np.zeros((1, n_ch, 160))
for j in tqdm(SList):
    name = 's' + str(j) + '_test.npy'
    a = np.load(name)
    x_test = np.concatenate((x_test, a), axis=0)
    y = (int(j) - 1) * np.ones((np.size(a, 0), 1))
    y_test = np.concatenate((y_test, y))
    # print(j)
    del a
    del y

# deleting one unwanted index
x_train = x_train[1:np.size(x_train, 0), :, :]
x_test = x_test[1:np.size(x_test, 0), :, :]
y_train = y_train[1:np.size(y_train, 0), :]
y_test = y_test[1:np.size(y_test, 0), :]

# data shape
print('Shape of x_train:', np.shape(x_train))
print('Shape of x_test:', np.shape(x_test))
print('Shape of y_train:', np.shape(y_train))
print('Shape of y_test:', np.shape(y_test))
input_shape = (x_train.shape[1], x_train.shape[2], 1)
print('Input shape:', input_shape)

# normalizing your data
maxMat = np.amax(x_train, axis=0, keepdims=True)
x_train = np.divide(x_train, maxMat)
x_test = np.divide(x_test, maxMat)
del maxMat

# shuffling data
shuffle = list(range(np.size(x_train, 0)))
np.random.shuffle(shuffle)
x_train = x_train[shuffle, :, :]
y_train = y_train[shuffle, ]

shuffle = list(range(np.size(x_test, 0)))
np.random.shuffle(shuffle)
x_test = x_test[shuffle, :, :]
y_test = y_test[shuffle, ]

print('Shape of x_train after shuffling:', np.shape(x_train))
print('Shape of x_test after shuffling:', np.shape(x_test))
print('Shape of y_train after shuffling:', np.shape(y_train))
print('Shape of y_test after shuffling:', np.shape(y_test))
input_shape = (x_train.shape[1], x_train.shape[2], 1)
print('Input shape after shuffling:', input_shape)

del shuffle

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2],
                          1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

num_classes = 109
y_train = np.reshape(y_train, [
    np.size(y_train, 0),
])
y_train = to_categorical(y_train, num_classes)
y_test = np.reshape(y_test, [
    np.size(y_test, 0),
])
y_test = to_categorical(y_test, num_classes)

print('Shape of x_train after reshaping:', np.shape(x_train))
print('Shape of x_test after reshaping:', np.shape(x_test))
print('Shape of y_train after reshaping:', np.shape(y_train))
print('Shape of y_test after reshaping:', np.shape(y_test))
input_shape = (x_train.shape[1], x_train.shape[2], 1)
print('Input shape after reshaping:', input_shape)

model = Sequential()

#layer1
model.add(
    Conv2D(128,
           kernel_size=(n_ch, 1),
           padding='valid',
           input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(
    Reshape(
        (model.output_shape[3], model.output_shape[2], model.output_shape[1])))

## Layer 2
model.add(Conv2D(256, kernel_size=(128, 1), padding='valid'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(
    Reshape(
        (model.output_shape[3], model.output_shape[2], model.output_shape[1])))

## Layer 3
model.add(Conv2D(512, kernel_size=(256, 1), padding='valid'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(
    Reshape(
        (model.output_shape[3], model.output_shape[2], model.output_shape[1])))

model.add(Flatten())

#output layer
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=lr,
                                                        decay=decay),
              metrics=['accuracy', 'FalsePositives', 'FalseNegatives'])

# Train the model
history = model.fit(x_train,
                    y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=shuffle,
                    verbose=1,
                    validation_split=0.2,
                    callbacks=[tensorboard_callback, checkpoint_callback],
                    initial_epoch=initial_epoch)

# Save the final model
model.save(final_model_path)

# Evaluate the model
score_train = model.evaluate(x_train, y_train, verbose=0)
score_test = model.evaluate(x_test, y_test, verbose=0)
print('mean train accuracy is:   ', (score_train[1]))
print('mean test accuracy is:   ', (score_test[1]))
print('FAR is:   ', (score_train[2] + score_test[2]) /
      (np.size(y_train, 0) + np.size(y_test, 0)))
print('FRR is:   ', (score_train[3] + score_test[3]) /
      (np.size(y_train, 0) + np.size(y_test, 0)))

# Plot and save accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid(True)
plt.savefig(acc_plot_path, dpi=600)
plt.show()

# Plot and save loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid(True)
plt.savefig(loss_plot_path, dpi=600)
plt.show()
# Save history data to a JSON file


Extracting EDF parameters from D:\EEG-ESRT\datasets\EEGMMIDB\S001\S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\EEG-ESRT\datasets\EEGMMIDB\S001\S001R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\EEG-ESRT\datasets\EEGMMIDB\S001\S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\EEG-ESRT\datasets\EEGMMIDB\S001\S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\EEG-ESRT\datasets\EEGMMIDB\S001\S001R05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\EEG-ESRT\datasets\EEGMMIDB\S001\S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF p